Requires openpyxl

In [5]:
import sys, os, os.path
if not 'EXPERIMENT_HOME' in os.environ: # Configure local paths if not already defined
    ML_HOME = os.path.abspath(os.path.join("..", "ml_home")) # ML workspace
    EXPERIMENT_HOME = os.path.abspath(os.path.join(ML_HOME, "..")) # Experiment workspace
    DATA_HOME = os.path.abspath(os.path.join(EXPERIMENT_HOME,"dataset")) # Dataset location
    os.environ["EXPERIMENT_HOME"] = EXPERIMENT_HOME
    os.environ["DATA_HOME"] = DATA_HOME
    sys.path.insert(0, EXPERIMENT_HOME)
    sys.path.insert(0, ML_HOME) # Add to path so can load our library
    os.chdir(EXPERIMENT_HOME) # Change working directory to experiment workspace
print("Experiment Home: ", os.path.abspath(os.curdir), "; Data Home:", DATA_HOME)

Experiment Home:  /home/jovyan ; Data Home: /home/jovyan/dataset


In [6]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///dataset/stock.db', echo=False)
engine.table_names()

['auth_group',
 'auth_group_permissions',
 'auth_permission',
 'auth_user',
 'auth_user_groups',
 'auth_user_user_permissions',
 'dataset_set100',
 'dataset_set50',
 'dataset_stock',
 'dataset_stock_set100_membership_periods',
 'dataset_stock_set50_membership_periods',
 'dataset_timeseries',
 'django_admin_log',
 'django_content_type',
 'django_migrations',
 'django_session',
 'litreview_literature',
 'litreview_literature_refgroups',
 'litreview_refgroup',
 'sqlite_sequence']

In [7]:
import sqlite3
import pandas as pd

DB_FILE = 'dataset/stock.db'
cnx = sqlite3.connect(DB_FILE)
# pd.read_sql_query("SELECT * FROM dataset_set50", cnx)

## List of SET100 updates

The index had 15 updates during the study period of 2015-2019. These are the records from update_id=Update_3 to update_id=Update_17.

In [8]:
# cnx.commit()
# cnx.close()
df_uplist = pd.read_sql("SELECT update_id AS update_seq, date_from, date_to, date_published FROM dataset_set100", cnx)
df_uplist

,update_seq,date_from,date_to,date_published
0,Update_1,2014-01-01,2014-06-30,2013-12-16
1,Update_2,2014-07-01,2014-12-31,2020-11-24
2,Update_3,2015-01-01,2015-06-30,2020-11-24
3,Update_4,2015-01-01,2015-06-30,2015-04-29
4,Update_5,2015-07-01,2015-12-31,2015-06-16
5,Update_6,2016-01-01,2016-06-30,2015-12-14
6,Update_7,2016-01-01,2016-06-30,2015-12-30
7,Update_8,2016-07-01,2016-12-31,2016-06-17
8,Update_9,2016-07-01,2016-12-31,2016-12-01
9,Update_10,2017-01-01,2017-06-30,2016-12-16


In [9]:
import json
addlist = json.loads(open("dataset/set100_add_list.json", "r").read())
df_addlist = pd.DataFrame()
for k, v in addlist.items():
    inc = ",".join(v)
    print(k,inc)
    df_addlist = df_addlist.append({"update_seq": k, "stocks_added": inc}, ignore_index=True)
# df_addlist

Update_3 
Update_4 IFEC
Update_5 BA,UNIQ,WHA,CKP,RS,MONO,CBG,LHBANK,BEAUTY,ASP,U,SEPPE,S
Update_6 SCN,GL,PTG,BLA,PLANB,PLAT,SAMTEL,GPSC,SCCC,VNG,TASCO,TU,CHG,WORK,EPG
Update_7 BDMS,SGP,BEM
Update_8 TRC,MTLS,ERW,IFEC,GLOBAL,JWD,COM7,BCH,TVO
Update_9 PSH
Update_10 SCN,SUPER,KAMART,VIBHA,SPRC,THANI,BIG,TKN
Update_11 BJC,RATCH,BPP,MONO,ANAN,GFPT,MEGA,SCCC,MALEE,BCPG,WORK,EA,PTL
Update_12 SGP,GGC,TPIPP,JMART,STA,UV,ORI,ESSO,PSL,WHAUP,MC,JWD,HANA
Update_13 RATCH,BLA,TOA,RS,DELTA,PRM,BGRIM,ERW,THANI,GLOW,TTW
Update_14 PLANB,ANAN,GULF,AEONTS,GOLD,MBK
Update_15 BEC
Update_16 OSP,JMT,JAS
Update_17 AWC


Stocks in Update_3


In [10]:
df_up3 = pd.read_sql("SELECT constituents FROM dataset_set100 WHERE update_id='Update_3'", cnx)
up3_stocks = df_up3[0:1].values[0][0].replace(" ",",")
print("Update_3 " + up3_stocks)

Update_3 AAV,ADVANC,AMATA,ANAN,AOT,AP,BANPU,BAY,BBL,BCH,BCP,BEC,BECL,BGH,BH,BIGC,BJC,BJCHI,BLAND,BMCL,BTS,CENTEL,CK,CPALL,CPF,CPN,DELTA,DEMCO,DTAC,EARTH,EGCO,ERW,GFPT,GLOBAL,GLOW,GUNKUL,HANA,HEMRAJ,HMPRO,ICHI,INTUCH,IRPC,ITD,IVL,JAS,KBANK,KCE,KKP,KTB,KTIS,KTC,LH,LOXLEY,LPN,M,MAJOR,MC,MEGA,MINT,NOK,PS,PSL,PTG,PTT,PTTEP,PTTGC,QH,RATCH,ROBINS,SAMART,SAWAD,SCB,SCC,SCCC,SF,SGP,SIM,SIRI,SPALI,SPCG,STA,STEC,STPI,SVI,TCAP,THAI,THCOM,THREL,TICON,TISCO,TMB,TOP,TPIPL,TRUE,TTA,TTCL,TTW,TUF,UV,VGI


In [19]:
up = {}
for i in range(3,18):
    df_up = pd.read_sql("SELECT constituents FROM dataset_set100 WHERE update_id='Update_%s'" % i, cnx)
    stocks = set(df_up[0:1].values[0][0].replace(" ",",").split(','))
    up[i] = stocks
up

{3: {'AAV',
  'ADVANC',
  'AMATA',
  'ANAN',
  'AOT',
  'AP',
  'BANPU',
  'BAY',
  'BBL',
  'BCH',
  'BCP',
  'BEC',
  'BECL',
  'BGH',
  'BH',
  'BIGC',
  'BJC',
  'BJCHI',
  'BLAND',
  'BMCL',
  'BTS',
  'CENTEL',
  'CK',
  'CPALL',
  'CPF',
  'CPN',
  'DELTA',
  'DEMCO',
  'DTAC',
  'EARTH',
  'EGCO',
  'ERW',
  'GFPT',
  'GLOBAL',
  'GLOW',
  'GUNKUL',
  'HANA',
  'HEMRAJ',
  'HMPRO',
  'ICHI',
  'INTUCH',
  'IRPC',
  'ITD',
  'IVL',
  'JAS',
  'KBANK',
  'KCE',
  'KKP',
  'KTB',
  'KTC',
  'KTIS',
  'LH',
  'LOXLEY',
  'LPN',
  'M',
  'MAJOR',
  'MC',
  'MEGA',
  'MINT',
  'NOK',
  'PS',
  'PSL',
  'PTG',
  'PTT',
  'PTTEP',
  'PTTGC',
  'QH',
  'RATCH',
  'ROBINS',
  'SAMART',
  'SAWAD',
  'SCB',
  'SCC',
  'SCCC',
  'SF',
  'SGP',
  'SIM',
  'SIRI',
  'SPALI',
  'SPCG',
  'STA',
  'STEC',
  'STPI',
  'SVI',
  'TCAP',
  'THAI',
  'THCOM',
  'THREL',
  'TICON',
  'TISCO',
  'TMB',
  'TOP',
  'TPIPL',
  'TRUE',
  'TTA',
  'TTCL',
  'TTW',
  'TUF',
  'UV',
  'VGI'},
 4: {'AAV',
  '

In [21]:
up[4].difference(up[3]) # Added in 3

{'IFEC'}

In [22]:
up[3].difference(up[4]) # Removed in 3

{'HEMRAJ'}

In [23]:
up[16].difference(up[17])

{'BEAUTY'}

In [25]:
len(up[17])

100

In [15]:
df_addlist.loc[df_addlist["update_seq"]=='Update_3','stocks_added'] = up3_stocks
df_addlist[["update_seq", "stocks_added"]]

,update_seq,stocks_added
0,Update_3,"AAV,ADVANC,AMATA,ANAN,AOT,AP,BANPU,BAY,BBL,BCH..."
1,Update_4,IFEC
2,Update_5,"BA,UNIQ,WHA,CKP,RS,MONO,CBG,LHBANK,BEAUTY,ASP,..."
3,Update_6,"SCN,GL,PTG,BLA,PLANB,PLAT,SAMTEL,GPSC,SCCC,VNG..."
4,Update_7,"BDMS,SGP,BEM"
5,Update_8,"TRC,MTLS,ERW,IFEC,GLOBAL,JWD,COM7,BCH,TVO"
6,Update_9,PSH
7,Update_10,"SCN,SUPER,KAMART,VIBHA,SPRC,THANI,BIG,TKN"
8,Update_11,"BJC,RATCH,BPP,MONO,ANAN,GFPT,MEGA,SCCC,MALEE,B..."
9,Update_12,"SGP,GGC,TPIPP,JMART,STA,UV,ORI,ESSO,PSL,WHAUP,..."


In [4]:
# Union all stocks

In [28]:
union_set = set()
for r in df_addlist["stocks_added"]:
    # print(r.split(","))
    union_set.update(r.split(","))
len(union_set)

163

In [34]:
df_stocks = pd.DataFrame(sorted(list(union_set)), columns=['ticker'])
df_stocks.index = range(1, len(df_stocks) + 1)
df_stocks

,ticker
1,AAV
2,ADVANC
3,AEONTS
4,AMATA
5,ANAN
...,...
159,VIBHA
160,VNG
161,WHA
162,WHAUP


# Save Output



In [18]:
!pip install 

     |████████████████████████████████| 245kB 2.6MB/s eta 0:00:01
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-cp37-none-any.whl size=8914 sha256=282c4ede37262129390c97fd49bc82973d9282d4a85a021af1fe879d9fbea0a9
  Stored in directory: /home/jovyan/.cache/pip/wheels/2a/77/35/0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built et-xmlfile


In [20]:
# See documentation https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html

with pd.ExcelWriter("SET100.xlsx") as writer:
    df_uplist.to_excel(writer, sheet_name="1-UpdateSeq")
    df_addlist[["update_seq", "stocks_added"]].to_excel(writer, sheet_name="2-StocksAdded")
    df_stocks.to_excel(writer, sheet_name="3-AllStocks")